In [28]:
from sklearn.linear_model import LinearRegression
import numpy as np
from scipy import stats

In [2]:
%run "data_preperation.ipynb"

<>:1: SyntaxWarning: invalid escape sequence '\B'
C:\Users\arnet\AppData\Local\Temp\ipykernel_9852\4088575790.py:1: SyntaxWarning: invalid escape sequence '\B'
  file_path = 'data\BB_Rates_15min_from_10-2023.xlsx'


preperation complete!


C:\Users\arnet\AppData\Local\Temp\ipykernel_9852\2991408273.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  EURUSD_filtered['TimeFrame'] = EURUSD_filtered.groupby('Block').cumcount() + 1


In [43]:
def fit_model(block_data):
    X = np.arange(96).reshape(-1, 1)  # Independent variable (time steps)
    Y = block_data["Close"].values  # Dependent variable (closing prices)

    # Create and fit the model
    model = LinearRegression()
    model.fit(X, Y)
    return model

In [56]:
def predict_CI(model, confidence_level = 0.8):
    predictions = model.predict(X)
    # Predict the closing prices for t = 97 with confidence interval
    t = 97
    prediction = model.predict([[t]])[0]

    # Calculate the confidence interval
    n = len(X)
    t_value = stats.t.ppf((1 + confidence_level) / 2, n - 2)
    s = np.sqrt(np.sum((Y - predictions) ** 2) / (n - 2))
    s_x = np.sum((X - np.mean(X)) ** 2)
    s_y = s ** 2 * (1 + 1 / n + (t - np.mean(X)) ** 2 / s_x)
    confidence_interval = t_value * np.sqrt(s_y)
    return (prediction - confidence_interval), (prediction + confidence_interval)

In [118]:
count = 0
count_total = 0
trades = 0
bank = 5000000
for block in EURUSD_filtered["Block"].unique():
    block_data = EURUSD_filtered[EURUSD_filtered["Block"] == block]
    mutex = 0
    for t in range(96, len(block_data)):
        count_total += 1
        # fit model to the last 96 closing prices of the block
        block_data_last_day = block_data.iloc[t-96:t]
        model = fit_model(block_data_last_day)
        # predict the closing price CI for t = 97
        lower_bound, upper_bound = predict_CI(model, confidence_level = 0.1)

        mutex, bank = revert_to_the_mean(block_data.iloc[t], model, mutex, bank)

        # if lower_bound > block_data.iloc[t]["Close"] or upper_bound < block_data.iloc[t]["Close"]:
            # count += 1
            # print("Block:", block, "t:", t, "Lower bound:", lower_bound, "Upper bound:", upper_bound, "Actual closing price:", block_data.iloc[t]["Close"])
    if mutex == 1:
        bank = bank * (1/block_data.iloc[t]["Close"])
        print("Sell", (1/block_data.iloc[t]["Close"]) * bank, "€", block_data.iloc[t]["Dates"], block_data.iloc[t]["Close"], "end of block")
        

Buy 5253000.0 $ 2023-10-04 23:15:00 1.0506
Sell 4989551.6717325235 € 2023-10-05 03:00:00 1.0528
Buy 5250006.268996961 $ 2023-10-05 03:15:00 1.0522
Sell 4986234.465758345 € 2023-10-05 03:45:00 1.0529
Buy 5247014.528317506 $ 2023-10-05 04:15:00 1.0523
Sell 4985760.669248866 € 2023-10-05 13:30:00 1.0524
Buy 5237541.583045933 $ 2023-10-05 14:30:00 1.0505
Sell 4974868.524929648 € 2023-10-05 15:15:00 1.0528
Buy 5244506.398980835 $ 2023-10-06 02:45:00 1.0542
Sell 4966388.635398518 € 2023-10-06 12:00:00 1.056
Buy 5218184.539213223 $ 2023-10-06 14:30:00 1.0507
Sell 4945209.002286983 € 2023-10-06 16:30:00 1.0552
Buy 5225107.831816426 $ 2023-10-10 06:00:00 1.0566
Sell 4934001.729760554 € 2023-10-10 10:15:00 1.059
Buy 5219680.42991369 $ 2023-10-10 13:15:00 1.0579
Sell 4922369.322815627 € 2023-10-10 16:00:00 1.0604
Buy 5218695.9560491275 $ 2023-10-11 04:15:00 1.0602
Sell 4909865.421064191 € 2023-10-11 15:45:00 1.0629
Buy 5199547.480906977 $ 2023-10-11 19:30:00 1.059
Sell 4935498.320747011 € 2023-10

In [108]:
print("Percentage of closing prices outside the confidence interval:", count/count_total)
print("Bank:", bank)
print("Trades:", trades)

Percentage of closing prices outside the confidence interval: 0.0
Bank: 273219928.5554864
Trades: 0


In [115]:
def revert_to_the_mean(closing_price, model, mutex, bank):
    lower, upper = predict_CI(model, confidence_level = 0.1) # inner CI
    lower_lower, upper_upper = predict_CI(model, confidence_level = 0.8) # outer CI
    # we distinguish the two cases to take care of possible level shifts 
    if mutex == 0: # no investment made, potentially buy
        if closing_price["Close"] < lower and closing_price["Close"] > lower_lower:
            #trades += 1
            print("Buy", closing_price["Close"] * bank, "$", closing_price["Dates"], closing_price["Close"])
            return 1, bank * closing_price["Close"]
        else:
            return 0, bank 
    else: # investment made, potentially sell, mutex == 1
        if closing_price["Close"] > upper and closing_price["Close"] < upper_upper:
            print("Sell", (1/closing_price["Close"]) * bank, "€", closing_price["Dates"], closing_price["Close"])
            return 0, bank * (1/closing_price["Close"])
        else:
            return 1, bank
        